In [ ]:
def assign_patches_to_columns(
    left: np.ndarray,
    width: np.ndarray,
    col_left: np.ndarray,
    col_right: np.ndarray,
    page_width: int
) -> np.ndarray:
    """
    
    """
    center = left + width / 2

    boundaries = [0]
    for r, l in zip(col_right, col_left[1:]):
        boundaries.extend([r, l])
    boundaries.append(page_width)
    boundaries = np.array(boundaries)

    # Pair boundaries into columns
    column_starts = boundaries[::2]
    column_ends   = boundaries[1::2]

    # 3. Vectorized assignment
    # Broadcast: patch_centers[:,None] vs column_starts[None,:]
    inside = (center[:, None] >= column_starts[None, :]) & \
             (center[:, None] < column_ends[None, :])

    # argmax along columns axis gives the column id
    column_ids = np.argmax(inside, axis=1)

    return column_ids


In [ ]:

for page_number, page_df in patches_df.groupby('page'):
    page_width = (page_df['left'] + page_df['width']).max()
    page_file = page_df['file'].iloc[0]
    
    proj = np.zeros(page_width, dtype=np.uint16)

    for _, patch in page_df.iterrows():
        left = patch['left']
        w    = patch['width']
        proj[left:left+w] += 1
    
    from skimage.filters import threshold_otsu
    t = threshold_otsu(proj)


    cond = (proj <= t).astype(int)
    diff = cond[1:]-cond[:-1]
    left_cols = np.where(diff==-1)[0]
    right_cols = np.where(diff==1)[0]

    # plt.figure(figsize=(16,9))
    # plt.plot(proj)
    # plt.vlines(left_cols, ymin=0, ymax=20, colors="gold", linestyles='--', linewidths=2)
    # plt.vlines(right_cols, ymin=0, ymax=20, colors="green", linestyles='--', linewidths=2)

    # plt.show()

    image_folder = Path('data/datasets/book1')
    img = Image.open(image_folder / page_file)

    # fig = plt.figure(figsize=(16, 9))
    # plt.imshow(img)
    # plt.axis("off")

    # h = img.height

    # plt.vlines(left_cols, ymin=0, ymax=h, colors="gold", linestyles="--", linewidth=2)
    # plt.vlines(right_cols, ymin=0, ymax=h, colors="green", linestyles="--", linewidth=2)

    # plt.close(fig)

    # savefig(fig, 'linedetection')

    column_ids = assign_patches_to_columns(
        page_df['left'].values,
        page_df['width'].values,
        left_cols,
        right_cols,
        page_width
    )

    patches_df.loc[page_df.index, 'column_id'] = column_ids
    page_df['column_id'] = column_ids

    i = 0
    for column, column_df in page_df.groupby('column_id', sort=True):
        order = column_df['top'].argsort()
        patches_df.loc[column_df.index, 'in_column_order'] = order
        patches_df.loc[column_df.index, 'reading_order'] = order+i
        i += len(column_df)


    # == VIZ ==

    image_folder = Path('data/datasets/book1')
    img = Image.open(image_folder / page_file)

    fig, ax = plt.subplots(figsize=(16, 9))
    ax.imshow(img)
    ax.axis("off")

    # Draw columns
    h = img.height
    # ax.vlines(left_cols, ymin=0, ymax=h, colors="gold", linestyles="--", linewidth=2)
    # ax.vlines(right_cols, ymin=0, ymax=h, colors="green", linestyles="--", linewidth=2)

    # Draw patch rectangles + numbers
    for idx, patch in page_df.iterrows():
        x, y = patch['left'], patch['top']
        w, hh = patch['width'], patch['height']
        rect = plt.Rectangle((x, y), w, hh, fill=False, edgecolor='red', linewidth=1.5)
        ax.add_patch(rect)

        reading_num = int(patches_df.loc[idx, 'reading_order'])
        ax.text(x, y-2, str(reading_num), color='blue', fontsize=8, weight='bold')

    plt.show()



    break


In [ ]:


otsu = threshold_otsu(proj)



ymax = t * 1.2
plt.figure(figsize=(16,9))
plt.plot(proj)
plt.vlines(left_cols, ymin=0, ymax=ymax, colors="orange", linestyles='--', linewidths=2)
plt.vlines(right_cols, ymin=0, ymax=ymax, colors="green", linestyles='--', linewidths=2)
plt.hlines(y=[otsu], xmin=0, xmax=len(proj), colors="black", linestyles='--', linewidths=2, label='Otsu Threshold')
plt.hlines(y=[t], xmin=0, xmax=len(proj), colors="red", linestyles='--', linewidths=2, label='Manual Threshold')

plt.show()

for l, r in zip(left_cols, right_cols):
    # y-proj of the column
    column = (craft_comp._labels[:, l:r] != 0).sum(1)

    plt.plot(column)
    plt.show()
    plt.imshow(craft_comp._labels[:, l:r] != 0)
    break

# inside = (center[:, None] >= column_starts[None, :]) & \
#             (center[:, None] < column_ends[None, :])

# # argmax along columns axis gives the column id
# column_ids = np.argmax(inside, axis=1)



